In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
data = pd.read_csv('EOCC_Data_Robot_Input.csv')

In [3]:
#pca
pca1 = PCA(n_components=1)
pc_d_l = principalComponents = pca1.fit_transform(data[['CAR_AGE IN DAYS','LIFE TO DATE MILEAGE']])
data['PCA_d_l']=pc_d_l
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [4]:
#pca
pca1 = PCA(n_components=1)
pc_w_m = principalComponents = pca1.fit_transform(data[['Mileage since Last Replacement','COMPONENT_AGE IN DAYS']])
data['PCA_w_m']=pc_w_m
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [5]:
data.head()

,CAR NUMBER,LOCATION,Last Replacement Date,CURRENT_CONDITION_CODE,CUT_OFF_DATE,SHOP_INITIAL,Pocket_Type,Pocket_Number,Mileage since Last Replacement,CAR_CLASS,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,3/23/2018,C_1,3/6/2019,UP,0,NaN,21436,GSH64,...,0,12,20.329629,1060.389629,14.625925,581.204074,348,11934,222167.543748,-338011.264039
1,83500,B,6/6/2018,C_1,3/6/2019,UP,0,NaN,15750,GSH64,...,0,12,20.329629,1060.389629,14.625925,581.204074,273,11934,222167.543748,-343697.757931
2,83503,A,12/19/2014,C_3,3/6/2019,UP,0,NaN,69043,GSH64,...,0,19,30.188571,1083.472571,16.408571,802.380571,1538,11929,220313.572853,-290392.999194
3,83503,B,10/20/2016,C_3,3/6/2019,UP,0,NaN,30207,GSH64,...,0,19,30.188571,1083.472571,16.408571,802.380571,867,11929,220313.572853,-329234.386184
4,83505,A,6/22/2017,C_1,3/6/2019,UP,0,NaN,22552,GSH64,...,0,28,17.542857,1087.278367,17.858333,861.054375,622,11929,269908.601790,-336891.878117


In [6]:
data['CURRENT_CONDITION_CODE'] = pd.factorize(data.CURRENT_CONDITION_CODE)[0]
data['SHOP_INITIAL'] = pd.factorize(data.SHOP_INITIAL)[0]
data['Pocket_Type'] = pd.factorize(data.Pocket_Type)[0]
data['Pocket_Number'] = pd.factorize(data.Pocket_Number)[0]
data['CAR_CLASS'] = pd.factorize(data.CAR_CLASS)[0]
data['Equipment_Feature_Desc'] = pd.factorize(data.Equipment_Feature_Desc)[0]
data['BUILDER_NAME'] = pd.factorize(data.BUILDER_NAME)[0]

In [7]:
data.head()

,CAR NUMBER,LOCATION,Last Replacement Date,CURRENT_CONDITION_CODE,CUT_OFF_DATE,SHOP_INITIAL,Pocket_Type,Pocket_Number,Mileage since Last Replacement,CAR_CLASS,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,3/23/2018,0,3/6/2019,0,0,-1,21436,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,348,11934,222167.543748,-338011.264039
1,83500,B,6/6/2018,0,3/6/2019,0,0,-1,15750,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,273,11934,222167.543748,-343697.757931
2,83503,A,12/19/2014,1,3/6/2019,1,0,-1,69043,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,1538,11929,220313.572853,-290392.999194
3,83503,B,10/20/2016,1,3/6/2019,0,0,-1,30207,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,867,11929,220313.572853,-329234.386184
4,83505,A,6/22/2017,0,3/6/2019,0,0,-1,22552,0,...,0,28,17.542857,1087.278367,17.858333,861.054375,622,11929,269908.601790,-336891.878117


In [8]:
best_vals_reg=[]
for j in range(15):
    data_unique_car_number = pd.DataFrame(data['CAR NUMBER'].unique())
    data_unique_car_number.columns=['CAR NUMBER']
    
    data_random=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_random.columns=['RandomNum']
    
    data_unique_car_number['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    
    data_merged =pd.merge(data,data_unique_car_number,on='CAR NUMBER',how='left')

    data_new = data_merged.drop(['CAR NUMBER','LOCATION', 'Last Replacement Date','CUT_OFF_DATE','BUILT DATE'],axis =1)
    
    #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)
    
    #ada = ADASYN(random_state=42)
    #x_train, y_train = ada.fit_resample(x_train, y_train)
    
    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20,                             
                                    n_jobs = -1).fit(x_train,y_train)
    
    vals=[]
    for i in np.linspace(0.01,0.2,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals),np.linspace(0.01,0.2,50)[vals.index(max(vals))]))
    best_vals_reg.append(max(vals)) 


0.6634310025777568-0.037142857142857144
0.6833631621617486-0.037142857142857144
0.6707870439540577-0.025510204081632654
0.6615368465832967-0.025510204081632654
0.6779427549194991-0.03326530612244898
0.6573339171470969-0.029387755102040815
0.6805884383940022-0.029387755102040815
0.6621322871746176-0.037142857142857144
0.6492201932729071-0.029387755102040815
0.6497646447895205-0.03326530612244898
0.6846504991130038-0.03326530612244898
0.6687456204612052-0.025510204081632654
0.636939665682317-0.025510204081632654
0.6764665876246811-0.03326530612244898
0.649064425770308-0.04489795918367347


In [9]:
np.mean(best_vals_reg)

0.6647978059750678

In [9]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
for i in range(10):
    print(list(x_train.columns.values)[indices[i]])

COMPONENT_AGE IN DAYS
CAR_AGE IN DAYS
Mileage since Last Replacement
LIFE TO DATE MILEAGE
AVG_Loaded_Trip_Duration_Days
AVG_Loaded_Distance_Miles
AVG_Empty_Trip_Duration_Days
AVG_Empty_Distance_Miles
Loading_Count
Mountain Central


In [63]:
x_train.corr()['CAR_AGE IN DAYS']['LIFE TO DATE MILEAGE']

0.8777858632355616